# Imports

In [1]:
import os
import cv2
import numpy as np
from PIL import Image
from datetime import datetime
import threading
import tkinter as tk
from tkinter import messagebox
import os
import csv

# Name utils

In [2]:
def add_name(name):
    users_file = "usersdb.txt"

    # Determine the next person_id
    if not os.path.exists(users_file):
        with open(users_file, "w") as user_file:
            pass

    with open(users_file, "r") as user_file:
        lines = user_file.readlines()
        next_id = len(lines) + 1

    # Append to names file
    with open(users_file, "a") as user_file:
        user_file.write(f"{next_id},{name}\n")

    return next_id


def get_name_by_id(person_id):
    users_file = "usersdb.txt"

    with open(users_file, "r") as user_file:
        for line in user_file:
            key_value = line.strip().split(',')
            if len(key_value) == 2 and int(key_value[0]) == person_id:
                return key_value[1]

    return "Not found"

# Capture faces for train

In [3]:
def capture_faces(name):
    face_cascade = cv2.CascadeClassifier('pretrained/haarcascade_frontalface_alt2.xml')

    # Get new person person_id and name
    person_id = add_name(name)
    photo_n = 0
    img_capt = cv2.VideoCapture(0)

    # Take 50 photos
    while photo_n < 50:
        ret, eyes_img = img_capt.read()
        grey_img = cv2.cvtColor(eyes_img, cv2.COLOR_BGR2GRAY)

        # Checking that the illumination is medium
        if np.average(grey_img) > 110:
            # Finds face on image with 1.3 scaleFactor and 5 minNeighbors
            detected_faces = face_cascade.detectMultiScale(grey_img, 1.3, 5)
            # x, y - top left corner coordinates of face and w, h - face width and height
            for (x, y, w, h) in detected_faces:
                # Crop with a little margin around the edges
                cropped_img = grey_img[y - int(h / 2): y + int(h * 1.5), x - int(x / 2): x + int(w * 1.5)]

                # Try to detect and straighten eye_list
                eyes_img = detect_eyes(cropped_img)

                if eyes_img is not None:
                    # Aligned to eye_list image
                    cur_frame = eyes_img
                else:
                    cur_frame = grey_img[y: y + h, x: x + w]

                # Save captured face
                cv2.imwrite(f"photos/User.{person_id}.{photo_n}.jpg", cur_frame)
                cv2.imshow("Captured photo", cur_frame)
                cv2.moveWindow("Captured photo", 1100, 50)
                photo_n += 1
                cv2.waitKey(300)

        cv2.imshow('Face Recognition System Capture detected_faces', grey_img)
        cv2.moveWindow('Face Recognition System Capture detected_faces', 450, 50)

    print('Face capture is complete')
    img_capt.release()
    cv2.destroyAllWindows()


# Rotate image based on eyes

In [4]:
def detect_eyes(image):
    eye_cascade = cv2.CascadeClassifier('pretrained/haarcascade_eye.xml')
    # Searches for eye_list in the image and returns a list of rectangles with eye coordinates
    eye_list = eye_cascade.detectMultiScale(image)

    if len(eye_list) == 2:
        # Calculate eye_angle and rotate if needed
        (x1, y1, w1, h1), (x2, y2, w2, h2) = eye_list

        # Calculate rotation angle based on eye positions
        center_x1 = x1 + w1 // 2
        center_y1 = y1 + h1 // 2
        center_x2 = x2 + w2 // 2
        center_y2 = y2 + h2 // 2

        dx = center_x2 - center_x1
        dy = center_y2 - center_y1

        eye_angle = np.degrees(np.arctan2(dy, dx))

        # Rotate image
        height, width = image.shape
        rotation_matrix = cv2.getRotationMatrix2D((width / 2, height / 2), eye_angle, 1)
        rotated_image = cv2.warpAffine(image, rotation_matrix, (width, height))

        return rotated_image

    return None

# Train LBPH recognizer on new images

In [5]:
def train_recognizer():
    # 1,1 - radius and n of neighbours and 7,7 is image subdivision grid
    trained_rec = cv2.face.LBPHFaceRecognizer_create(1, 1, 7, 7)
    photo_path = 'photos'

    def get_images_with_id(photo_path):
        photo_paths = [os.photo_path.join(photo_path, f) for f in os.listdir(photo_path)]
        faces_list = []
        id_list = []

        for imagePath in photo_paths:
            cur_face = Image.open(imagePath).convert('L')
            cur_face = cur_face.resize((150, 150))
            cur_face_array = np.array(cur_face, 'uint8')

            cur_id = int(os.photo_path.split(imagePath)[-1].split('.')[1])
            faces_list.append(cur_face_array)
            id_list.append(cur_id)

        return np.array(id_list), faces_list

    id_list, faces_list = get_images_with_id(photo_path)

    print('Training')
    trained_rec.train(faces_list, id_list)

    # Ensure directory exists
    os.makedirs('traindata', exist_ok=True)

    trained_rec.save('traindata/trainingLBPH.xml')
    print('XML saved')

# Write to log recognized persons

In [6]:
def recognize_and_attendance(filename):
    face_cascade = cv2.CascadeClassifier('pretrained/haarcascade_frontalface_alt2.xml')
    trained_rec = cv2.face.LBPHFaceRecognizer_create()
    trained_rec.read("traindata/trainingLBPH.xml")
    if os.path.exists(filename + ".csv"):
        filename = filename + "(1)"
    cap = cv2.VideoCapture(0)
    file_name = filename + "_log.txt"
    # Ensure attendance log exists
    if not os.path.exists(file_name):
        open(file_name, "w").close()

    # Set to keep track of recognized people to avoid duplicate entries
    recognized_persons = set()

    while True:
        r, cur_frame = cap.read()
        grey_img = cv2.cvtColor(cur_frame, cv2.COLOR_BGR2GRAY)

        # Finds face on image with 1.3 scaleFactor and 5 minNeighbors
        detected_faces = face_cascade.detectMultiScale(grey_img, 1.3, 5)

        for (x, y, w, h) in detected_faces:
            cropped_img = grey_img[y:y + h, x:x + w]

            try:
                person_id, confidence = trained_rec.predict(cropped_img)

                # Threshold for confidence value
                if confidence < 30:
                    person_name = get_name_by_id(person_id)

                    # Check if person already recognized in this session
                    if person_name not in recognized_persons:
                        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

                        # Log attendance
                        with open(file_name, "a") as log_file:
                            print(person_name + " was recognized at " + timestamp)
                            log_file.write(f"{person_name},{timestamp}\n")

                        recognized_persons.add(person_name)

                    # Draw rectangle and person_name
                    cv2.rectangle(cur_frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    cv2.putText(cur_frame, f"{person_name} ({confidence:.2f})",
                                (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9,
                                (0, 255, 0), 2)
            except Exception as e:
                print(f"Error: {e}")

        cv2.imshow('Attendance System', cur_frame)
        cv2.moveWindow('Attendance System', 500, 50)

        # Break loop if session ended (file with attendance at the created)
        if cv2.waitKey(1) & (os.path.exists(filename + ".csv")):
            break

    cap.release()
    cv2.destroyAllWindows()

# Main app

In [7]:
def start_face_recognition(file_name):
    recognition_thread = threading.Thread(target=recognize_and_attendance, args=(file_name,))
    recognition_thread.daemon = True
    recognition_thread.start()


def end_attendance(window, file_name):
    # Move attendance log to CSV
    if os.path.exists(file_name + "_log.txt"):
        with open(file_name + "_log.txt", "r") as log_file, \
                open(file_name + ".csv", "w", newline='') as csv_file:
            csv_writer = csv.writer(csv_file)
            csv_writer.writerow(["Name", "Timestamp"])

            for line in log_file:
                name, timestamp = line.strip().split(",")
                csv_writer.writerow([name, timestamp])

        os.remove(file_name + "_log.txt")
        messagebox.showinfo("Success", f"Attendance saved to {file_name}")
    else:
        messagebox.showwarning("Warning", "No attendance records found.")

    window.destroy()


class AttendanceManagementSystem:
    def __init__(self, root):
        self.root = root
        self.root.title("Attendance Management System")
        self.root.geometry("400x300+50+50")

        # Create directories if they don't exist
        os.makedirs("photos", exist_ok=True)
        os.makedirs("traindata", exist_ok=True)

        # Create Main Menu
        self.create_menu()

    def create_menu(self):
        # Add New Person Button
        add_person_btn = tk.Button(self.root, width=20, height=2,
                                   text="Add New Person", font="15", command=self.add_new_person)
        add_person_btn.pack(pady=20)

        # Take Attendance Button
        take_attendance_btn = tk.Button(self.root, width=20, height=2, font="15",
                                        text="Take Attendance", command=self.take_attendance)
        take_attendance_btn.pack(pady=20)

        # Close Button
        take_attendance_btn = tk.Button(self.root, width=20, height=2, font="15", text="Close", command=self.close_app)
        take_attendance_btn.pack(pady=20)

    def close_app(self):
        self.root.destroy()

    def add_new_person(self):
        # Open a new window for adding a person
        add_person_window = tk.Toplevel(self.root)
        add_person_window.title("Add New Person")
        add_person_window.geometry("400x300+50+50")

        # Enter name
        tk.Label(add_person_window, text="Enter Name:", font=("Arial", 12)).pack(pady=10)
        name_entry = tk.Entry(add_person_window, font=("Arial", 12))
        name_entry.pack(pady=10)

        def capture_and_train():
            name = name_entry.get()
            if name:
                capture_faces(name)  # Pass name to face_capture
                train_recognizer()
                messagebox.showinfo("Success", f"New person '{name}' added and trained successfully!")
                add_person_window.destroy()
            else:
                messagebox.showerror("Error", "Name cannot be empty!")

        tk.Button(add_person_window, text="Capture & Train", command=capture_and_train, font=("Arial", 12)).pack(
            pady=20)

    def take_attendance(self):
        # Open attendance taking window
        attendance_window = tk.Toplevel(self.root)
        attendance_window.title("Take Attendance")
        attendance_window.geometry("400x300+50+50")

        filename = "attendance(" + datetime.now().strftime("%d-%m-%Y_%H-%M") + ")"

        # Start attendance button
        start_btn = tk.Button(attendance_window, text="Start Attendance", width=20, height=2, font="15",
                              command=lambda: start_face_recognition(filename))
        start_btn.pack(pady=20)

        # End attendance button
        end_btn = tk.Button(attendance_window, text="End Attendance", width=20, height=2, font="15",
                            command=lambda: end_attendance(attendance_window, filename))
        end_btn.pack(pady=20)


# Main app running

In [ ]:
root = tk.Tk()
AttendanceManagementSystem(root)
root.mainloop()